# Experimental Design: From New Haven to Microtargeting

**Class 7 | Feb 11 | Persuasion at Scale**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Persuasion-at-Scale/causal-inference-intro/blob/main/experimental-design.ipynb)

**Readings:** Issenberg, *The Victory Lab* (Prologue, Ch 1, 3, 5); Gerber & Green, *Field Experiments* (Ch 1)

Last time we saw *why* randomization works (potential outcomes, unbiasedness). Today we explore *how* experiments work in practice:

1. **The New Haven experiment** - multi-arm trial, estimating effects of canvassing, phone, and mail
2. **Statistical power** - why some effects are detectable and others aren't
3. **Confounding in action** - when observational data points in the wrong direction
4. **Microtargeting** - using predictive models to find the right voters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

BLUE = '#0072B2'
ORANGE = '#D55E00'
GREEN = '#009E73'
RED = '#c0392b'
GRAY = '#888888'
np.random.seed(42)

---
## 1. The New Haven Experiment

In 1998, Green and Gerber randomly assigned 30,000 New Haven voters to one of four conditions:

| Condition | Description | True effect (from the paper) |
|:---|:---|:---:|
| **Canvassing** | Yale students knock on doors, $20/hr | **+8.7 pp** |
| **Phone calls** | Commercial call center in North Dakota | **+0.0 pp** |
| **Direct mail** | Postcards (civic duty, community, close-election) | **+0.6 pp** |
| **Control** | No contact | - |

Let's simulate this. We'll generate voters with a baseline probability of turning out, apply the treatment effects, and see what the experiment reveals.

In [ ]:
# Simulate the New Haven experiment
N = 30000
true_effects = {'Control': 0.0, 'Canvass': 0.087, 'Phone': 0.0, 'Mail': 0.006}

# Baseline turnout probability for each voter (varies across population)
baseline_logit = np.random.normal(-0.5, 1.0, N)  # logit scale
baseline_prob = 1 / (1 + np.exp(-baseline_logit))  # average ~38% turnout

# Random assignment: equal probability for each arm
arms = ['Control', 'Canvass', 'Phone', 'Mail']
assignment = np.random.choice(arms, size=N)

# Generate outcomes
effect = np.array([true_effects[a] for a in assignment])
prob_vote = np.clip(baseline_prob + effect, 0, 1)
voted = np.random.binomial(1, prob_vote)

df = pd.DataFrame({
    'assignment': assignment,
    'voted': voted,
    'baseline_prob': baseline_prob
})

print(f"Total voters: {N:,}")
print(f"Per arm: ~{N//4:,}")
print(f"Overall turnout: {voted.mean():.1%}")

In [ ]:
# Estimate treatment effects (difference vs control)
control_rate = df.loc[df['assignment'] == 'Control', 'voted'].mean()

results = []
for arm in ['Canvass', 'Phone', 'Mail']:
    arm_data = df.loc[df['assignment'] == arm, 'voted']
    ctrl_data = df.loc[df['assignment'] == 'Control', 'voted']
    
    est = arm_data.mean() - ctrl_data.mean()
    se = np.sqrt(arm_data.var(ddof=1)/len(arm_data) + ctrl_data.var(ddof=1)/len(ctrl_data))
    ci_lo = est - 1.96 * se
    ci_hi = est + 1.96 * se
    
    results.append({
        'Treatment': arm,
        'Turnout': f"{arm_data.mean():.1%}",
        'Estimated effect (pp)': f"{est*100:+.1f}",
        'True effect (pp)': f"{true_effects[arm]*100:+.1f}",
        '95% CI': f"[{ci_lo*100:+.1f}, {ci_hi*100:+.1f}]",
        'Significant?': 'Yes' if abs(est/se) > 1.96 else 'No'
    })

results_df = pd.DataFrame(results)
print(f"Control turnout: {control_rate:.1%}\n")
display(results_df)

In [ ]:
# Visualize: estimated effects with confidence intervals
fig, ax = plt.subplots(figsize=(10, 5))

arm_names = ['Canvass', 'Phone', 'Mail']
colors = [GREEN, RED, BLUE]

for i, arm in enumerate(arm_names):
    arm_data = df.loc[df['assignment'] == arm, 'voted']
    ctrl_data = df.loc[df['assignment'] == 'Control', 'voted']
    est = arm_data.mean() - ctrl_data.mean()
    se = np.sqrt(arm_data.var(ddof=1)/len(arm_data) + ctrl_data.var(ddof=1)/len(ctrl_data))
    
    ax.errorbar(est * 100, i, xerr=1.96*se*100, fmt='o', color=colors[i],
                markersize=12, capsize=8, capthick=2, linewidth=2)
    # True effect marker
    ax.plot(true_effects[arm]*100, i, 's', color=colors[i], markersize=10,
            markeredgecolor='black', markeredgewidth=1.5, alpha=0.5)

ax.axvline(0, color='black', linewidth=1, linestyle='-')
ax.set_yticks(range(3))
ax.set_yticklabels(arm_names, fontsize=14)
ax.set_xlabel('Effect on turnout (percentage points)', fontsize=13)
ax.set_title('New Haven Experiment: Treatment Effects\n(circles = estimates with 95% CI, squares = true effects)', fontsize=13)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

print("Canvassing has a clear, large effect.")
print("Phone calls: indistinguishable from zero.")
print("Mail: tiny effect, hard to detect even with 30,000 voters.")

### Discussion

Notice that the **mail effect** (+0.6 pp) is real but almost impossible to detect, even with ~7,500 voters per arm. The confidence interval is wide relative to the effect.

This raises a critical question: *how many subjects do you need to reliably detect an effect of a given size?* That's the question of **statistical power**.

---
## 2. Statistical Power

**Power** = the probability that your experiment correctly detects a real effect (rejects H0 when H0 is false).

Power depends on three things:
- **Effect size**: bigger effects are easier to detect
- **Sample size (N)**: more data = more precision
- **Variance**: noisier outcomes require more data

The conventional target is 80% power: you want at least an 80% chance of detecting the effect if it's real.

Let's compute power curves for the three New Haven treatments.

In [ ]:
def compute_power(effect_size, n_per_arm, baseline_rate=0.38, n_sims=2000):
    """Simulate experiments and compute the fraction that reject H0."""
    significant = 0
    for _ in range(n_sims):
        control = np.random.binomial(1, baseline_rate, n_per_arm)
        treated = np.random.binomial(1, baseline_rate + effect_size, n_per_arm)
        diff = treated.mean() - control.mean()
        se = np.sqrt(treated.var(ddof=1)/n_per_arm + control.var(ddof=1)/n_per_arm)
        if se > 0 and abs(diff / se) > 1.96:
            significant += 1
    return significant / n_sims

# Power curves for each effect size
sample_sizes = [100, 250, 500, 1000, 2500, 5000, 7500, 10000, 25000, 50000]
effects = {
    'Canvass (+8.7 pp)': 0.087,
    'Mail (+0.6 pp)': 0.006,
    'Phone (+0.0 pp)': 0.001,  # tiny effect to show flat line
}

print("Computing power curves (this takes ~30 seconds)...")
power_results = {}
for label, eff in effects.items():
    powers = [compute_power(eff, n) for n in sample_sizes]
    power_results[label] = powers
    print(f"  {label}: done")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors_power = {'Canvass (+8.7 pp)': GREEN, 'Mail (+0.6 pp)': BLUE, 'Phone (+0.0 pp)': RED}

for label, powers in power_results.items():
    ax.plot(sample_sizes, powers, 'o-', color=colors_power[label], label=label,
            linewidth=2, markersize=6)

ax.axhline(0.8, color=GRAY, linestyle='--', linewidth=1.5, label='80% power target')
ax.axhline(0.05, color=GRAY, linestyle=':', linewidth=1, alpha=0.5)

ax.set_xscale('log')
ax.set_xlabel('Sample size per arm', fontsize=13)
ax.set_ylabel('Power (probability of detecting the effect)', fontsize=13)
ax.set_title('Power Curves: How Many Voters Do You Need?', fontsize=14)
ax.set_ylim(-0.02, 1.05)
ax.legend(fontsize=12, loc='center right')
ax.set_xticks(sample_sizes)
ax.set_xticklabels([f"{n:,}" for n in sample_sizes], rotation=45)

plt.tight_layout()
plt.show()

print("Canvassing: detectable with just a few hundred voters per arm.")
print("Mail: you need TENS OF THOUSANDS to reliably detect a 0.6 pp effect.")
print("This is why Green & Gerber needed 30,000 voters.")

### The power lesson

Campaigns spend billions on tactics with *tiny* effect sizes. Detecting those effects requires *enormous* experiments. This is why political science went decades without field experiments: the logistics of randomizing thousands of voters seemed impossible.

> "No one has any idea of the value of the ad versus a phone call from a friend." - Samuel Popkin

---
## 3. Confounding in Action

Gerber & Green Ch 1 emphasize that correlations can be "a misleading guide to causality." Here's a classic example from medicine:

**Which hospital is better?** Hospital A (a community hospital) has a 95% survival rate. Hospital B (a major teaching hospital) has only 85% survival. Should you go to Hospital A?

Not so fast. Hospital B treats the sickest patients - complex surgeries, rare cancers, emergency transfers. Hospital A mostly handles routine cases. The *patient mix* is a confounder: it drives both hospital choice and outcomes.

Let's simulate this and see how badly observational data misleads us.

In [ ]:
# Simulate the hospital confounding example
N_patients = 1000

# Patient severity (the confounder) - higher = sicker
severity = np.random.exponential(3, N_patients)

# Hospital selection: sicker patients go to Hospital B (the teaching hospital)
# This is NOT random - it's driven by severity
p_hospital_B = 1 / (1 + np.exp(-(severity - 4)))  # sicker -> more likely B
went_to_B = np.random.binomial(1, p_hospital_B)

# TRUE EFFECT of Hospital B: IMPROVES survival by 5 percentage points
# (better doctors, better equipment, more resources)
TRUE_HOSPITAL_EFFECT = 0.05

# Survival depends on severity (negatively) and hospital quality (positively)
baseline_survival = 1 / (1 + np.exp(0.5 * severity - 2))  # sicker -> lower survival
survival_prob = np.clip(baseline_survival + TRUE_HOSPITAL_EFFECT * went_to_B, 0, 1)
survived = np.random.binomial(1, survival_prob)

patients = pd.DataFrame({
    'severity': severity,
    'hospital': np.where(went_to_B, 'B (teaching)', 'A (community)'),
    'went_to_B': went_to_B,
    'survived': survived
})

# Naive observational comparison
surv_A = patients.loc[patients['went_to_B']==0, 'survived'].mean()
surv_B = patients.loc[patients['went_to_B']==1, 'survived'].mean()
obs_diff = surv_B - surv_A

print("Observational comparison:")
print(f"  Hospital A (community) survival: {surv_A:.1%}")
print(f"  Hospital B (teaching)  survival: {surv_B:.1%}")
print(f"  Naive estimate (B - A):          {obs_diff:+.1%}")
print(f"  True causal effect of B:         {TRUE_HOSPITAL_EFFECT:+.1%}")
print(f"\nThe naive estimate has the WRONG SIGN!")
print(f"Hospital B looks WORSE, but it's actually BETTER.")
print(f"The confounder (patient severity) drives both hospital choice and outcomes.")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: The misleading observational picture
ax = axes[0]
colors_h = [RED if b else BLUE for b in patients['went_to_B']]
ax.scatter(patients['severity'], patients['survived'] + np.random.normal(0, 0.02, N_patients),
           c=colors_h, alpha=0.3, s=30)
ax.scatter([], [], c=BLUE, label='Hospital A (community)', s=60)
ax.scatter([], [], c=RED, label='Hospital B (teaching)', s=60)

# Show severity distributions
ax2 = ax.twinx()
sev_A = patients.loc[patients['went_to_B']==0, 'severity']
sev_B = patients.loc[patients['went_to_B']==1, 'severity']
ax2.hist(sev_A, bins=20, alpha=0.2, color=BLUE, density=True)
ax2.hist(sev_B, bins=20, alpha=0.2, color=RED, density=True)
ax2.set_ylabel('Density of patients', fontsize=10, color=GRAY)
ax2.tick_params(axis='y', labelcolor=GRAY)

ax.set_xlabel('Patient severity', fontsize=12)
ax.set_ylabel('Survived (0/1, jittered)', fontsize=12)
ax.set_title('Selection bias: sicker patients go to Hospital B', fontsize=12)
ax.legend(fontsize=11, loc='center right')

# Panel 2: What a randomized trial would show
ax = axes[1]
random_B = np.random.binomial(1, 0.5, N_patients)
survival_prob_rct = np.clip(baseline_survival + TRUE_HOSPITAL_EFFECT * random_B, 0, 1)
survived_rct = np.random.binomial(1, survival_prob_rct)

rct_A = survived_rct[random_B == 0].mean()
rct_B = survived_rct[random_B == 1].mean()
rct_diff = rct_B - rct_A

x_pos = [0, 1]
bars = ax.bar(x_pos, [surv_A, surv_B], color=[BLUE, RED], width=0.35,
              label='Observational', alpha=0.4)
bars2 = ax.bar([x + 0.35 for x in x_pos], [rct_A, rct_B], color=[BLUE, RED], width=0.35,
               label='Randomized', edgecolor='black', linewidth=1.5)

ax.set_xticks([0.175, 1.175])
ax.set_xticklabels(['Hospital A', 'Hospital B'], fontsize=13)
ax.set_ylabel('Survival rate', fontsize=12)
ax.set_title(f'Observational: B looks worse (diff={obs_diff:+.1%})\n'
             f'Randomized: B is actually better (diff={rct_diff:+.1%})', fontsize=12)
ax.legend(fontsize=11)
ax.set_ylim(0, 1.1)

# Annotate
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{bar.get_height():.0%}', ha='center', fontsize=11, color=GRAY)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{bar.get_height():.0%}', ha='center', fontsize=11, fontweight='bold')

plt.suptitle('Confounding: Why Observational Data Can Point in the Wrong Direction',
             fontsize=14, fontweight='bold', y=1.03)
plt.tight_layout()
plt.show()

### The basketball analogy

> "Basketball players tend to be taller than other people, but you cannot grow taller by joining the basketball team." - Gerber & Green

Hospital B has lower survival rates - but you cannot worsen your health by going there. The hospital and the outcome share a common cause (patient severity), and the correlation between hospital choice and survival reflects that common cause, not the hospital's quality.

Gerber & Green call the list of possible confounders "a bottomless pit" with "no well-defined stopping rule." You can try to control for severity, but what about insurance status, distance to hospital, time of day, referring physician...?

This is precisely why Fisher's insight was so important: **random assignment** severs the link between treatment and all confounders, observed or not.

---
## 4. Microtargeting: Finding the Right Voters

Issenberg (Ch 5) describes Alex Gage's breakthrough: merging consumer data (Acxiom) with voter files to predict political leanings from shopping behavior.

> "HBO subscription predicted openness to Romney." - Issenberg

The idea: instead of targeting by geography (all voters in a precinct), target by individual. Build a model that predicts who's persuadable, then concentrate resources on those people.

Let's simulate this. We'll generate a population of voters with consumer attributes and political preferences, build a simple model, and see how targeted outreach compares to broadcast.

In [ ]:
# Simulate voter population with consumer data
N_voters = 5000

# Consumer variables (a la Acxiom)
income = np.random.lognormal(10.5, 0.6, N_voters)  # household income
has_hbo = np.random.binomial(1, 0.3, N_voters)
college_grad = np.random.binomial(1, 0.35, N_voters)
attends_church = np.random.binomial(1, 0.25, N_voters)
drives_pickup = np.random.binomial(1, 0.15, N_voters)
reads_nyt = np.random.binomial(1, 0.12, N_voters)
age = np.random.normal(45, 15, N_voters).clip(18, 90)

# Latent political leaning (higher = more conservative)
# These weights are made up but directionally plausible
leaning = (
    0.3 * (income > 80000).astype(float)
    - 0.4 * has_hbo
    + 0.5 * attends_church
    + 0.6 * drives_pickup
    - 0.5 * reads_nyt
    - 0.3 * college_grad
    + 0.01 * (age - 45)
    + np.random.normal(0, 0.8, N_voters)
)

# Binary: supports our candidate (say, a moderate Republican like Romney 2002)
p_support = 1 / (1 + np.exp(-leaning))
supports_candidate = np.random.binomial(1, p_support)

voters = pd.DataFrame({
    'income': income,
    'has_hbo': has_hbo,
    'college_grad': college_grad,
    'attends_church': attends_church,
    'drives_pickup': drives_pickup,
    'reads_nyt': reads_nyt,
    'age': age,
    'supports_candidate': supports_candidate
})

print(f"Voter population: {N_voters:,}")
print(f"Support rate: {supports_candidate.mean():.1%}")
print(f"\nConsumer variable correlations with support:")
features = ['has_hbo', 'college_grad', 'attends_church', 'drives_pickup', 'reads_nyt']
for feat in features:
    rate_1 = voters.loc[voters[feat]==1, 'supports_candidate'].mean()
    rate_0 = voters.loc[voters[feat]==0, 'supports_candidate'].mean()
    print(f"  {feat:20s}: {rate_1:.1%} vs {rate_0:.1%}  (diff: {rate_1-rate_0:+.1%})")

In [ ]:
# Build a predictive model (like Gage did for Bush 2004)
feature_cols = ['has_hbo', 'college_grad', 'attends_church', 'drives_pickup', 'reads_nyt', 'age']
X = voters[feature_cols].values
y = voters['supports_candidate'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Predicted probabilities
voters['predicted_support'] = model.predict_proba(X)[:, 1]

auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f"Model AUC: {auc:.3f}\n")

print("Feature importances (logistic regression coefficients):")
for feat, coef in sorted(zip(feature_cols, model.coef_[0]), key=lambda x: -abs(x[1])):
    direction = 'more likely' if coef > 0 else 'less likely'
    print(f"  {feat:20s}: {coef:+.3f}  ({direction} to support)")

In [ ]:
# Compare targeted vs untargeted outreach
# Scenario: you can canvass 500 voters (10% of population)
# Canvassing effect: +8.7 pp turnout among supporters
BUDGET = 500
CANVASS_EFFECT = 0.087

# Baseline turnout (everyone has ~50% chance of voting)
voters['baseline_turnout'] = np.random.uniform(0.3, 0.7, N_voters)

# Strategy 1: Random (broadcast) - canvass 500 random voters
random_targets = np.random.choice(N_voters, BUDGET, replace=False)
random_contacted_supporters = voters.iloc[random_targets]['supports_candidate'].sum()

# Strategy 2: Targeted - canvass the 500 voters most likely to be supporters
targeted_idx = voters['predicted_support'].nlargest(BUDGET).index
targeted_contacted_supporters = voters.iloc[targeted_idx]['supports_candidate'].sum()

# Expected additional votes from each strategy
# (only supporters who are contacted AND whose turnout is flipped by canvassing)
random_extra_votes = random_contacted_supporters * CANVASS_EFFECT
targeted_extra_votes = targeted_contacted_supporters * CANVASS_EFFECT

print("Canvassing budget: 500 door knocks")
print("="*50)
print(f"\nRandom targeting:")
print(f"  Supporters contacted: {random_contacted_supporters} / {BUDGET}")
print(f"  Expected extra votes: {random_extra_votes:.1f}")
print(f"\nMicrotargeting:")
print(f"  Supporters contacted: {targeted_contacted_supporters} / {BUDGET}")
print(f"  Expected extra votes: {targeted_extra_votes:.1f}")
print(f"\nMicrotargeting advantage: {targeted_extra_votes/random_extra_votes:.1f}x more effective")

In [ ]:
# Visualize the targeting advantage
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Distribution of predicted support scores
ax = axes[0]
ax.hist(voters.loc[voters['supports_candidate']==1, 'predicted_support'],
        bins=30, alpha=0.6, color=BLUE, label='Actual supporters', density=True)
ax.hist(voters.loc[voters['supports_candidate']==0, 'predicted_support'],
        bins=30, alpha=0.6, color=ORANGE, label='Non-supporters', density=True)
threshold = voters['predicted_support'].nlargest(BUDGET).min()
ax.axvline(threshold, color='black', linestyle='--', linewidth=2,
           label=f'Top {BUDGET} threshold')
ax.set_xlabel('Predicted support probability', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Model separates supporters from non-supporters', fontsize=12)
ax.legend(fontsize=11)

# Panel 2: Efficiency comparison
ax = axes[1]
strategies = ['Random\n(broadcast)', 'Microtargeted']
supporter_rates = [
    random_contacted_supporters / BUDGET * 100,
    targeted_contacted_supporters / BUDGET * 100
]
bars = ax.bar(strategies, supporter_rates, color=[GRAY, GREEN], width=0.5)
for bar, rate in zip(bars, supporter_rates):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{rate:.0f}%', ha='center', fontsize=14, fontweight='bold')
ax.set_ylabel('% of contacted voters who are supporters', fontsize=12)
ax.set_title('Microtargeting concentrates resources on supporters', fontsize=12)
ax.set_ylim(0, 105)

plt.suptitle('"Without the ability to find 3 people out of 10 on a block,\nwe would have had to knock on all 10 doors." - Matthew Dowd',
             fontsize=12, style='italic', y=1.05)
plt.tight_layout()
plt.show()

---
## 5. The Full Picture: Experiments + Microtargeting

The arc of the readings:

1. **Experiments** tell you *what works* (canvassing >> phone calls)
2. **Microtargeting** tells you *who to reach* (find likely supporters from consumer data)
3. **The combination** is modern evidence-based campaigning

Let's put it together: what if you could run an experiment *within* each microtargeting segment?

In [ ]:
# Heterogeneous treatment effects: canvassing works better on some voters
# Suppose persuadable voters (predicted support ~0.4-0.6) respond MORE to canvassing

N_exp2 = 2000
predicted_scores = voters['predicted_support'].values[:N_exp2]

# Treatment effect varies: highest for "on the fence" voters
# (those with predicted support near 0.5)
persuadability = 1 - 4 * (predicted_scores - 0.5)**2  # peaks at 0.5
individual_effect = 0.15 * persuadability  # up to 15 pp for most persuadable

# Run an experiment
treat_exp = np.random.binomial(1, 0.5, N_exp2)
baseline_turnout = np.random.uniform(0.3, 0.6, N_exp2)
turnout = np.random.binomial(1, np.clip(baseline_turnout + individual_effect * treat_exp, 0, 1))

# Estimate effects by predicted support tercile
exp_df = pd.DataFrame({
    'predicted_support': predicted_scores,
    'treated': treat_exp,
    'voted': turnout,
    'persuadability': persuadability
})
exp_df['segment'] = pd.qcut(exp_df['predicted_support'], 3,
                             labels=['Unlikely supporter', 'Persuadable', 'Likely supporter'])

print("Treatment effects by voter segment:\n")
for seg in ['Unlikely supporter', 'Persuadable', 'Likely supporter']:
    seg_data = exp_df[exp_df['segment'] == seg]
    treat_rate = seg_data.loc[seg_data['treated']==1, 'voted'].mean()
    ctrl_rate = seg_data.loc[seg_data['treated']==0, 'voted'].mean()
    effect = treat_rate - ctrl_rate
    n = len(seg_data)
    print(f"  {seg:22s}  (N={n:4d}):  effect = {effect:+.1%}")

overall = exp_df.loc[exp_df['treated']==1, 'voted'].mean() - exp_df.loc[exp_df['treated']==0, 'voted'].mean()
print(f"\n  {'Overall':22s}  (N={N_exp2:4d}):  effect = {overall:+.1%}")
print("\nThe overall average masks variation across segments.")
print("Campaigns that can identify persuadable voters get more bang per door knock.")

---
## Summary

| Concept | Key takeaway |
|:---|:---|
| **Multi-arm experiments** | Different treatments have vastly different effects (8.7 pp vs 0 pp) |
| **Statistical power** | Small effects need large experiments - this is why campaign experiments were rare before 1998 |
| **Confounding** | Observational data can point in the WRONG DIRECTION (hospital example) |
| **Microtargeting** | Predictive models from consumer data concentrate resources on the right voters |
| **The combination** | Experiments identify what works; microtargeting identifies who to reach |

**Next:** HW2 - designing your own experiment